## preparation

In [1]:
import pandas as pd
import numpy as np
import scipy.optimize
import random
import os

In [2]:
fares = pd.read_csv('Fares.csv', sep = ';')
flights = pd.read_csv('Flights.csv', sep = ';')

In [3]:
history = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_001.csv', sep = ';')

In [4]:
history = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_001.csv', sep = ';')
history['id'] = '001'
for i in range(2,101,1):
    if len(str(i)) == 1:
        i = '00' + str(i)
    elif len(str(i)) == 2:
        i = '0' + str(i)
    else:
        i = str(i)
        
        
    hist = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_' + i + '.csv', 
                       sep = ';')
    hist['id'] = i
    history = history.append(hist)
    
    
   

In [5]:
def get_demands(fare_id, history):
    filt_history = history.loc[history.FARE_ID == fare_id,]
    demands = filt_history['id'].value_counts()
    return demands

def get_capacity(flight_id):
    return flights.loc[flights.FLIGHT_ID == flight_id]['CAPACITY']
    
def get_price(fare_id):
    return fares.loc[fares.FARE_ID == fare_id]['PRICE']

def get_flights(fare_id):
    dep_air  = int(fares.loc[fares.FARE_ID == fare_id]['DEPARTURE_AIRPORT'])
    dest_air = int(fares.loc[fares.FARE_ID == fare_id]['DESTINATION_AIRPORT'])
   
    if dep_air == 1 or dest_air == 1:
        return [flights.loc[(flights.DEPARTURE_AIRPORT == dep_air) &
                           (flights.DESTINATION_AIRPORT == dest_air)]['FLIGHT_ID']]
    else:
        return [flights.loc[(flights.DEPARTURE_AIRPORT == dep_air)]['FLIGHT_ID'], 
                flights.loc[(flights.DESTINATION_AIRPORT == dest_air)]['FLIGHT_ID']]




    

In [6]:
flight_fare_df = pd.DataFrame(index=range(flights.shape[0]),columns=range(fares.shape[0]), data = 0)
for i in range(fares.shape[0]):
    used_flights = get_flights(i+1)
    try: 
        flight_fare_df.iloc[int(used_flights[0]) - 1, i] = 1
        flight_fare_df.iloc[int(used_flights[1]) - 1, i] = 1
    except:
        flight_fare_df.iloc[int(used_flights[0]) - 1, i] = 1
        

## optimization

In [7]:
print(fares.columns, fares.shape)
print(flights.columns, flights.shape)
print(history.columns, history.shape)

Index(['FARE_ID', 'DEPARTURE_AIRPORT', 'DESTINATION_AIRPORT', 'CLASS',
       'PRICE'],
      dtype='object') (144, 5)
Index(['FLIGHT_ID', 'DEPARTURE_AIRPORT', 'DESTINATION_AIRPORT', 'CAPACITY'], dtype='object') (16, 4)
Index(['TIME', 'FARE_ID', 'id'], dtype='object') (24120, 3)


In [8]:
def kvadr_x(x):
    return (x ** 2 - 3 * x + 8) 

In [9]:
cons = ( {'type': 'ineq',
            'fun' : lambda x: np.array([-x, 0])})

In [10]:
x0 = np.array([100])
scipy.optimize.minimize(kvadr_x, x0,  method='SLSQP', constraints = cons)

     fun: 7.999999999999997
     jac: array([-3.])
 message: 'Optimization terminated successfully.'
    nfev: 10
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([8.8817842e-16])

In [11]:
capacity = flights['CAPACITY']
capacity = np.reshape(capacity, (16, 1))

C:\Users\StepanAsus\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:52: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [12]:
exp_demand = np.array(history['FARE_ID'].value_counts().sort_index()) / 100

In [13]:
exp_demand

array([3.38, 1.45, 1.76, 0.82, 4.16, 1.32, 0.03, 0.01, 3.64, 1.78, 2.51,
       0.78, 2.89, 1.23, 3.23, 1.32, 0.04, 0.03, 0.91, 0.35, 5.14, 2.32,
       2.1 , 1.01, 2.05, 0.91, 4.83, 2.06, 3.2 , 1.18, 0.78, 0.17, 5.14,
       2.31, 1.93, 0.73, 1.97, 0.78, 3.2 , 0.83, 3.87, 0.95, 2.5 , 0.99,
       2.44, 0.85, 3.6 , 1.15, 1.2 , 0.48, 3.01, 1.02, 3.26, 0.94, 3.81,
       1.54, 1.29, 0.56, 1.31, 0.37, 4.25, 1.31, 0.89, 0.35, 0.05, 0.01,
       1.03, 0.34, 3.89, 1.16, 2.68, 0.86, 0.83, 0.31, 4.2 , 1.7 , 3.65,
       1.28, 0.5 , 0.21, 0.13, 0.06, 0.48, 0.1 , 3.66, 1.55, 3.15, 1.18,
       1.91, 0.69, 1.73, 0.5 , 4.13, 1.39, 2.11, 0.8 , 3.8 , 1.2 , 0.26,
       0.08, 4.86, 1.56, 0.35, 0.11, 2.38, 0.74, 0.32, 0.06, 3.17, 1.3 ,
       4.88, 1.74, 1.32, 0.49, 1.9 , 0.52, 1.62, 0.71, 2.5 , 1.02, 2.39,
       0.99, 3.34, 1.38, 4.27, 1.27, 3.03, 1.08, 0.56, 0.14, 1.79, 0.53,
       3.92, 0.93, 1.17, 0.35, 0.92, 0.32, 2.08, 0.57, 3.69, 1.55, 3.9 ,
       1.65])

In [14]:
bounds = np.array([exp_demand[i] for i in range(len(exp_demand))]).reshape((144, 1))

In [15]:
bounds.shape

(144, 1)

In [16]:
A_ub=pd.DataFrame(np.vstack([np.array(flight_fare_df), np.identity(144)]))
b_ub=np.vstack([capacity, bounds])

In [17]:
allocations = scipy.optimize.linprog(c= - np.array(fares['PRICE']), A_ub=A_ub, 
                                     b_ub=b_ub)

In [18]:
allocations

     fun: -34721.03999999999
 message: 'Optimization terminated successfully.'
     nit: 137
   slack: array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.33, 0.  , 0.  , 0.  ,
       0.03, 0.  , 2.3 , 0.  , 2.51, 0.  , 1.61, 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 5.14, 0.  , 2.1 , 0.  , 2.05, 0.  , 1.79, 0.  ,
       0.  , 0.  , 0.  , 0.  , 1.6 , 0.  , 1.77, 0.  , 0.  , 0.  , 0.33,
       0.  , 0.  , 0.  , 2.5 , 0.  , 2.44, 0.  , 3.6 , 0.  , 0.  , 0.  ,
       3.01, 0.  , 0.17, 0.  , 3.81, 0.  , 1.29, 0.  , 1.31, 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 1.03, 0.  , 0.  , 0.  , 2.68, 0.  ,
       0.83, 0.31, 0.88, 0.  , 2.47, 0.  , 0.5 , 0.  , 0.  , 0.  , 0.48,
       0.  , 0.  , 0.  , 3.15, 0.  , 1.91, 0.69, 0.  , 0.  , 1.23, 0.  ,
       2.11, 0.  , 0.83, 0.  , 0.26, 0.  , 4.86, 0.  , 0.09, 0.  , 0.  ,
       0.  , 0.  , 0.  , 3.17, 0.  , 0.6 , 0.  , 0.  , 0.  , 0.  , 0.  ,
       1.62, 0.  , 0.

In [19]:
shadow_prices = scipy.optimize.linprog(c=b_ub.reshape((160,)), A_ub= - A_ub.T,
                                       b_ub= - np.array(fares['PRICE']).T)

In [20]:
shadow_prices.x.shape

(160,)

In [21]:
np.array(fares['PRICE'])

array([ 24, 192,  34, 272,  67, 536,  62, 496,  61, 488,  19, 152,  21,
       168,  47, 376,  24, 192,  53, 424,  47, 376,  56, 448,  56, 448,
        34, 272,  44, 352,  71, 568,  34, 272,  53, 424,  82, 656,  96,
       768,  95, 760,  19, 152,  19, 152,  44, 352,  67, 536,  47, 376,
        82, 656,  93, 744,  93, 744,  66, 528,  83, 664, 114, 912,  62,
       496,  56, 448,  96, 768,  93, 744,   2,  16,  82, 656,  79, 632,
        86, 688,  61, 488,  56, 448,  95, 760,  93, 744,   2,  16,  80,
       640,  77, 616,  84, 672,  19, 152,  34, 272,  19, 152,  66, 528,
        82, 656,  80, 640,  18, 144,  51, 408,  21, 168,  44, 352,  19,
       152,  83, 664,  79, 632,  77, 616,  18, 144,  32, 256,  47, 376,
        71, 568,  44, 352, 114, 912,  86, 688,  84, 672,  51, 408,  32,
       256], dtype=int64)

## bid prices

In [22]:
shadow_prices.x.shape

(160,)

In [23]:
thresholds = np.sum(np.multiply(flight_fare_df.T, 
                               shadow_prices.x[:16, np.newaxis].T), axis=1)

In [24]:
flight_fare_df.shape

(16, 144)

In [25]:
np.array(fares['PRICE']) >= thresholds

0       True
1       True
2       True
3       True
4       True
5       True
6       True
7       True
8       True
9       True
10     False
11      True
12      True
13      True
14      True
15      True
16      True
17      True
18      True
19      True
20     False
21      True
22     False
23      True
24     False
25      True
26      True
27      True
28      True
29      True
       ...  
114     True
115     True
116    False
117     True
118     True
119     True
120     True
121     True
122     True
123     True
124    False
125     True
126    False
127     True
128     True
129     True
130     True
131     True
132    False
133     True
134     True
135     True
136     True
137     True
138     True
139     True
140     True
141     True
142    False
143     True
Length: 144, dtype: bool

In [26]:
hist1 = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_001.csv', sep = ';')

In [27]:
np.array(flight_fare_df.iloc[:, 10]).shape

(16,)

In [28]:
(capacity - np.array(flight_fare_df.iloc[:, 10]).reshape((16, 1)))

array([[16],
       [21],
       [16],
       [14],
       [14],
       [17],
       [18],
       [15],
       [12],
       [20],
       [18],
       [14],
       [15],
       [20],
       [23],
       [16]], dtype=int64)

In [29]:
fares_available = np.array(fares['PRICE']) >= thresholds

In [30]:
fares_ID = [np.where(fares_available)[i] + 1 for i in range(len(np.where(fares_available)))]

In [31]:
fares_ID

[array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  13,  14,
         15,  16,  17,  18,  19,  20,  22,  24,  26,  27,  28,  29,  30,
         31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  44,
         46,  48,  49,  50,  52,  53,  54,  56,  58,  60,  61,  62,  63,
         64,  65,  66,  68,  69,  70,  72,  75,  76,  77,  78,  80,  81,
         82,  84,  85,  86,  88,  91,  92,  93,  94,  96,  97,  98, 100,
        102, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 115,
        116, 118, 119, 120, 121, 122, 123, 124, 126, 128, 129, 130, 131,
        132, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144], dtype=int64)]

In [32]:
fut = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\Future\Bookings_009.csv', sep = ';')

In [33]:
accepted_fares = [fut.FARE_ID.values[i] for i in range(len(fut.FARE_ID.values)) 
                  if fut.FARE_ID.values[i] in fares_ID[0]]

In [34]:
np.intersect1d(accepted_fares, fut.FARE_ID)

array([  1,   2,   3,   4,   5,   6,   9,  10,  12,  13,  15,  16,  19,
        24,  26,  27,  28,  29,  30,  31,  33,  34,  36,  37,  39,  41,
        42,  46,  48,  49,  52,  53,  56,  58,  60,  61,  63,  68,  69,
        70,  75,  76,  77,  85,  88,  91,  93,  94,  96,  97,  98, 103,
       105, 111, 112, 113, 114, 115, 118, 119, 120, 123, 126, 128, 129,
       131, 132, 134, 137, 140, 141, 142], dtype=int64)

In [35]:
np.array(fut.FARE_ID)

array([  9, 123,  93, 131,  43, 127, 141,  75,  71,  31,  53,  85, 109,
        21,  15, 133, 109,  69,   1,  55,  19,  69,  71,  53, 123, 111,
         3, 101,  61,  13,  95, 133, 141,  33, 101,  93,  41, 127,  15,
       133,  69, 131,  13,  91,  97,  71,  75, 123, 119,  51,  15, 111,
        47, 141,   5, 125,  31,  47,  55,  25,  91,  15,  75,  63,  39,
       113, 119,  27,  49,  69,  77,  55, 109,  33, 109,  11,  53,  11,
        37,   3,  69,  85,  69, 111,  69, 143,  21,  57,  93, 111, 143,
        61, 103,  45,  79,   3,  61, 101,  29, 143,  19,  47,  27, 133,
        47,  75,  45,  67,  39, 143,   3,  39,  27,   3,  87, 123,  87,
       137,  51,  39, 109,  87,  77,   5,  55,  43,  85,  55,  49, 101,
       129,  37, 101,  47,  45,  55, 125,  57,  15,  29,  93, 125, 141,
        61,  33,  57, 131,  28,  41,  71, 133,  21, 141, 119,  61,  85,
        85,  29,  26, 131,  26, 142,  36,  55, 109,   9,  10,  76,  98,
        13,  33,  39,  26,  48,  24, 125, 111,  52,  87,  71, 10

## DLP model

In [220]:
def bid_prices(fares, capacity, flight_fare_df, bookings, exp_demand):
    profit = 0
    bounds = np.array([exp_demand[i] for i in range(len(exp_demand))]).reshape((144, 1))
    A_ub = pd.DataFrame(np.vstack([np.array(flight_fare_df), np.identity(144)]))
    b_ub = np.vstack([capacity, bounds])
    
    for i in range(1, 7):
        shadow_prices = scipy.optimize.linprog(c=b_ub.reshape((160,)), A_ub= - A_ub.T,
                                               b_ub= - np.array(fares['PRICE']).T)
        thresholds = np.sum(np.multiply(A_ub.iloc[:16, :].T, 
                               shadow_prices.x[:16, np.newaxis].T), axis=1)

        bookings_aux = bookings[(bookings.TIME <= (i * 100)) & (bookings.TIME >= ((i - 1) * 100))]
        fares_available = np.array(fares['PRICE']) >= thresholds
        fares_ID = [np.where(fares_available)[k] + 1 for k in range(len(np.where(fares_available)))]
        accepted_fares = [bookings_aux.FARE_ID.values[k] for k in range(len(bookings_aux.FARE_ID.values))
                          if bookings_aux.FARE_ID.values[k] in fares_ID[0]]
        
        for j in accepted_fares:
            if np.sum(b_ub[:16] > 0) == 16:
                b_ub[:16] = b_ub[:16] - np.array(A_ub.iloc[:16, j - 1]).reshape((16, 1))
                profit += fares.iloc[j - 1, -1]
            elif np.sum((b_ub[:16] 
                            - np.array(A_ub.iloc[:16, j - 1]).reshape((16, 1))) >= 0) == 16:
                b_ub[:16] = b_ub[:16] - np.array(A_ub.iloc[:16, j - 1]).reshape((16, 1))
                profit += fares.iloc[j - 1, -1]
        print(profit, np.sum(b_ub[:16]))
            
    return profit  

In [170]:
def average_profit(directory):

    profit = 0
    path = 'C:/Users/StepanAsus/Documents/GitHub/Quant_Marketing_UvA/Ass_3/' + directory
    os.chdir(path)
    files = os.listdir(path)
    for file_ in files:
        d = pd.read_csv(file_, sep = ';')
        profit += bid_prices(fares, capacity, flight_fare_df, d, exp_demand)
    return profit / 100

In [110]:
files = os.listdir('C:/Users/StepanAsus/Documents/GitHub/Quant_Marketing_UvA/Ass_3/' + 'Future')
files[0]

'Bookings_001.csv'

In [138]:
average_profit('Future')

28549.54

In [221]:
bid_prices(fares, capacity, flight_fare_df, fut, exp_demand)

[10. 34. 48. 58. 56. 19. 16. 23. 19. 34. 47. 62. 61. 24. 21. 32.]
2135 220.0
[15. 34. 67. 58. 56.  4. 16. 32. 24. 38. 67. 62. 61. 19. 21. 47.]
3696 185.0
[15. 34. 67. 63. 56.  4. 16. 32. 24. 39. 67. 78. 61. 19. 21. 47.]
4812 153.0
[  0.  34.  67. 588.  33.   0.  21.  47.  24.  62.  67.  82. 595.  34.
  44.  51.]
5614 143.0
[  0.  34.  67. 588.  33.   0.  21.  47.  24.  62.  67.  82. 595.  47.
  44.  51.]
11336 108.0
[  0.  34. 587. 538.  26.  19. 198.  47. 192.  69.  67. 157. 726. 118.
  77.  58.]
23816 49.0


23816

## RLP model

In [159]:
zeros = pd.DataFrame(np.zeros((144, 1)))
indexes = fut['FARE_ID'].value_counts().sort_index()
output = zeros.join(indexes, how='left')['FARE_ID'].fillna(0)

In [272]:
random.choice(os.listdir('C:/Users/StepanAsus/Documents/GitHub/Quant_Marketing_UvA/Ass_3/History'))

'Bookings_057.csv'

In [287]:
def bid_prices_RLP(fares, capacity, flight_fare_df, bookings):
    profit = 0
    path_hist = 'C:/Users/StepanAsus/Documents/GitHub/Quant_Marketing_UvA/Ass_3/History'
    files_hist = os.listdir(path_hist)
    exp_demand = pd.DataFrame(np.zeros((144,)))
    
    for i in range(100):
        data = pd.read_csv(random.choice(os.listdir(path_hist)), sep = ';')
        zeros = pd.DataFrame(np.zeros((144, 1)))
        indexes = data['FARE_ID'].value_counts().sort_index()
        exp_demand += zeros.join(indexes, how='left')['FARE_ID'].fillna(0).values.reshape((144, 1))
        
    exp_demand = exp_demand / 100
    A_ub = pd.DataFrame(np.vstack([np.array(flight_fare_df), np.identity(144)]))
    b_ub = np.vstack([capacity, exp_demand])

    for i in range(1, 7):
        shadow_prices = scipy.optimize.linprog(c=b_ub.reshape((160,)), A_ub= - A_ub.T,
                                               b_ub= - np.array(fares['PRICE']).T)
        thresholds = np.sum(np.multiply(A_ub.iloc[:16, :].T, 
                               shadow_prices.x[:16, np.newaxis].T), axis=1)

        bookings_aux = bookings[(bookings.TIME <= (i * 100)) & (bookings.TIME >= ((i - 1) * 100))]
        fares_available = np.array(fares['PRICE']) >= thresholds
        fares_ID = [np.where(fares_available)[k] + 1 for k in range(len(np.where(fares_available)))]
        accepted_fares = [bookings_aux.FARE_ID.values[k] for k in range(len(bookings_aux.FARE_ID.values))
                          if bookings_aux.FARE_ID.values[k] in fares_ID[0]]
        
        for j in accepted_fares:
            if np.sum(b_ub[:16] > 0) == 16:
                b_ub[:16] = b_ub[:16] - np.array(A_ub.iloc[:16, j - 1]).reshape((16, 1))
                profit += fares.iloc[j - 1, -1]
            elif np.sum((b_ub[:16] 
                            - np.array(A_ub.iloc[:16, j - 1]).reshape((16, 1))) >= 0) == 16:
                b_ub[:16] = b_ub[:16] - np.array(A_ub.iloc[:16, j - 1]).reshape((16, 1))
                profit += fares.iloc[j - 1, -1]
#         print(profit, np.sum(b_ub[:16]))
            
    return profit

In [282]:
bid_prices_RLP(fares, capacity, flight_fare_df, fut)

24 269.0
64 267.0
64 267.0
1232 261.0
8556 225.0
18844 184.0


18844

In [285]:
def average_profit_RLP(directory):

    profit = 0
    path = 'C:/Users/StepanAsus/Documents/GitHub/Quant_Marketing_UvA/Ass_3/' + directory
    os.chdir(path)
    files = os.listdir(path)
    for file_ in files:
        d = pd.read_csv(file_, sep = ';')
        profit += bid_prices_RLP(fares, capacity, flight_fare_df, d)
    return profit / 100

In [286]:
average_profit_RLP('Future')

146 263.0
256 259.0
256 259.0
1795 253.0
10928 217.0
25800 156.0
0 270.0
24 269.0
187 262.0
686 257.0
7881 227.0
25089 157.0
44 268.0
99 266.0
123 265.0
1667 260.0
12784 211.0
28712 153.0
74 267.0
74 267.0
74 267.0
1154 262.0
9986 225.0
21802 183.0
0 270.0
79 266.0
124 264.0
1228 257.0
10396 220.0
23924 169.0
0 270.0
53 268.0
77 267.0
4134 249.0
12046 216.0
27670 158.0
83 266.0
131 264.0
131 264.0
1763 257.0
10747 222.0
25715 171.0
165 264.0
344 256.0
363 255.0
2347 246.0
14723 200.0
30867 148.0
24 269.0
64 267.0
64 267.0
1232 261.0
8556 225.0
19292 182.0
114 265.0
114 265.0
114 265.0
4010 250.0
12218 219.0
27690 158.0
44 268.0
176 262.0
254 258.0
1157 250.0
10021 219.0
26061 160.0
0 270.0
24 269.0
68 267.0
1105 262.0
14057 217.0
29777 162.0
24 269.0
24 269.0
64 267.0
872 263.0
13500 222.0
29892 160.0
121 266.0
161 264.0
195 263.0
3137 250.0
12945 213.0
25633 165.0
58 267.0
106 265.0
197 261.0
2514 249.0
14746 207.0
31314 149.0
114 265.0
114 265.0
182 263.0
1062 257.0
9830 223.0
29182 

26152.51